## Imports

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from google.cloud import bigquery
from pathlib import Path
import sys

sys.path.insert(0, "../")
from params import *

sys.path.insert(0, '../custom_functions')
from get_data_with_cache import get_data_with_cache

sys.path.insert(0, '../ml_logic')
from preprocessor import *
from data import *

In [6]:
# Import data from CSV (temp)
wingman_data = pd.read_csv("../raw_data/trimmed_data/wingman_data.csv")

In [ ]:
# ## Import data from GBQ
# query = f"""SELECT * FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_TABLE}"""
# data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("preclean", f"query_{DATA_SIZE}.csv")

# wingman_data = get_data_with_cache(query, data_query_cache_path)

In [7]:
# Clean
wingman_data_clean = clean_data(wingman_data)
# Preprocess
wingman_data_proc = preprocess_features(wingman_data_clean)

/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/data.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_cleaned.drop([
/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/encoders.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_enc[''] = np.where(wingman_data_enc['type_fly'].isin(top_9_categories), wingman_data_enc['type_fly'], 'Other')
/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/encoders.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

## Train test split

In [14]:
X = wingman_data_proc.drop(columns=["eventsoe_no"])
y = wingman_data_proc["eventsoe_no"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## Baseline model

In [16]:
baseline_mod = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=-1)
baseline_mod.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=1)

In [17]:
## baseline prediction

y_pred = baseline_mod.predict(X_test)

accuracy_score(y_test, y_pred)

0.14361525704809286

## Model performance (not sure if this is needed for baseline)

In [18]:
## confusion matrix

confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
## classification report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
          10       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         3
          30       0.00      0.00      0.00         1
          70       0.00      0.00      0.00         6
          80       0.00      0.00      0.00         6
          81       0.00      0.00      0.00         3
          82       0.00      0.00      0.00         0
          90       0.10      0.04      0.05        84
          91       0.00      0.00      0.00         2
          92       0.17      0.08      0.11       145
          93       0.00      0.00      0.00         8
          94       0.11      0.04      0.06        67
          95       0.06      0.05      0.05        41
          96       0.00      0.00      0.00        63
          97       0.00      0.00      0.00         4
         100       0.00      0.00      0.00         1
         110       0.00    

/home/aaron/.pyenv/versions/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aaron/.pyenv/versions/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aaron/.pyenv/versions/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [20]:
## feature importance

feature_imp = pd.Series(baseline_mod.feature_importances_, index = wingman_data.feature_names).sort_values(ascending = False)
feature_imp

AttributeError: 'DataFrame' object has no attribute 'feature_names'